In [1]:
import tensorflow as tf
import numpy as np
tf.executing_eagerly()  # 检查是否启用了即时执行模式


# 超参数
learning_rate = 1e-4
keep_prob_rate = 0.7 
max_epoch = 2000
batch_size = 100

# 加载 MNIST 数据集
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 定义模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (7, 7), activation='relu', input_shape=(28, 28, 1), padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2, padding='same'),
    tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(1 - keep_prob_rate),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 定义优化器和损失函数
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# 训练函数
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# 计算准确率的函数
def compute_accuracy(preds, labels):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, axis=1), 
                                          tf.argmax(labels, axis=1)), 
                                tf.float32))

# 训练循环
for epoch in range(max_epoch):
    # 随机选择一个 batch
    indices = np.random.permutation(len(x_train))[:batch_size]
    batch_x = x_train[indices]
    batch_y = y_train[indices]
    
    # 训练一步
    loss = train_step(batch_x, batch_y)
    
    # 每100次迭代打印一次准确率
    if epoch % 100 == 0:
        predictions = model(x_test[:1000], training=False)
        accuracy = compute_accuracy(predictions, y_test[:1000])
        print(f'Iteration {epoch}, Test Accuracy: {accuracy.numpy():.4f}')  # 使用 .numpy() 转换为浮点数

# 最终测试
predictions = model(x_test, training=False)
final_accuracy = compute_accuracy(predictions, y_test)
print(f'Final Test Accuracy: {final_accuracy.numpy():.4f}')  # 使用 .numpy() 转换为浮点数

Iteration 0, Test Accuracy: 0.1720
Iteration 100, Test Accuracy: 0.8720
Iteration 200, Test Accuracy: 0.9130
Iteration 300, Test Accuracy: 0.9290
